<a href="https://colab.research.google.com/github/maahieummah/30-Days-Map-Challenge/blob/main/srinagar_lal_chowk_to_dal_lake_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
"""
AI Auto-Route Detection + Layers + Animations — FINAL SCRIPT
Works in: Jupyter / Colab / VS Code / Terminal
Outputs: ai_routes_final.html (auto-open attempt)
"""

# ---------------------------
# Install missing packages (runs only if package is absent)
# ---------------------------
import sys, subprocess, importlib
def ensure(pkg, import_name=None):
    try:
        importlib.import_module(import_name or pkg)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

for pkg in ["osmnx", "networkx", "folium", "geopandas", "shapely", "pandas", "numpy"]:
    ensure(pkg)

# ---------------------------
# Imports
# ---------------------------
import os, webbrowser, math
import osmnx as ox
import networkx as nx
import folium
from folium.plugins import (MiniMap, MeasureControl, Fullscreen, MousePosition, TimestampedGeoJson)
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point, LineString

# ---------------------------
# User settings
# ---------------------------
PLACE = "Srinagar, Jammu and Kashmir, India"
ORIGIN = (34.083656, 74.797371)      # Lal Chowk (lat, lon)
DESTINATION = (34.136847, 74.837295) # Nigeen Lake (lat, lon)
OUTFILE = "ai_routes_final.html"
ANIMATION_ROUTE_CHOICE = "Fastest"   # "Shortest" / "Fastest" / "Scenic" / "Safe"

# ---------------------------
# 1) Download & prepare graph
# ---------------------------
print("Loading graph for:", PLACE)
G = ox.graph_from_place(PLACE, network_type="drive", simplify=True)
print("Nodes:", len(G.nodes), "Edges:", len(G.edges))

# Ensure length / speeds / travel_time exist
try:
    G = ox.add_edge_speeds(G)           # speed_kph
    G = ox.add_edge_travel_times(G)     # travel_time (seconds)
    print("Edge speeds & travel times added.")
except Exception as e:
    print("Warning: add_edge_speeds/travel_times failed, applying fallback speeds.")
    default_kph = 30.0
    for u,v,k,d in G.edges(keys=True, data=True):
        if "speed_kph" not in d:
            d["speed_kph"] = default_kph
        if "travel_time" not in d:
            speed_mps = d["speed_kph"] * 1000.0 / 3600.0
            d["travel_time"] = d.get("length",0.0) / max(speed_mps, 0.1)

# ---------------------------
# 2) Utility helpers
# ---------------------------
def nearest_node(point):
    # point: (lat, lon)
    return ox.distance.nearest_nodes(G, point[1], point[0])

def coords_from_nodes(route_nodes):
    return [(G.nodes[n]['y'], G.nodes[n]['x']) for n in route_nodes]

def route_metrics(route_nodes):
    total_len = 0.0
    total_time = 0.0
    for i in range(len(route_nodes)-1):
        u = route_nodes[i]; v = route_nodes[i+1]
        # take first key if multi-edges
        data = next(iter(G[u][v].values()))
        total_len += data.get("length", 0.0)
        total_time += data.get("travel_time", 0.0)
    return total_len, total_time

# ---------------------------
# 3) Define route computations (defensive)
# ---------------------------
def compute_route(start_node, end_node, weight_attr):
    try:
        return nx.shortest_path(G, start_node, end_node, weight=weight_attr)
    except nx.NetworkXNoPath:
        print(f"No path found with weight '{weight_attr}'")
        return []
    except Exception as e:
        print("Routing error:", e)
        return []

# Scenic: lightweight heuristic (penalize major highways)
def compute_scenic_route(start_node, end_node):
    # create weight function that penalizes motorway/trunk/primary
    def weight_fn(u, v, attrs):
        hw = attrs.get("highway")
        base = attrs.get("length", 1.0)
        if isinstance(hw, list):
            hw_val = hw[0]
        else:
            hw_val = hw
        if hw_val in ("motorway", "trunk", "primary"):
            return base * 3.0
        return base
    try:
        return nx.shortest_path(G, start_node, end_node, weight=weight_fn)
    except Exception as e:
        print("Scenic route error:", e)
        return []

# Safe: heuristic (penalize small service/residential)
def compute_safe_route(start_node, end_node):
    def weight_fn(u, v, attrs):
        hw = attrs.get("highway")
        base = attrs.get("length", 1.0)
        if isinstance(hw, list):
            hw_val = hw[0]
        else:
            hw_val = hw
        if hw_val in ("service","residential","unclassified","tertiary"):
            return base * 1.4
        if hw_val in ("motorway","trunk","primary"):
            return base * 0.9
        return base
    try:
        return nx.shortest_path(G, start_node, end_node, weight=weight_fn)
    except Exception as e:
        print("Safe route error:", e)
        return []

# ---------------------------
# 4) Compute nodes & routes
# ---------------------------
orig_node = nearest_node(ORIGIN)
dest_node = nearest_node(DESTINATION)
print("Origin node:", orig_node, "Destination node:", dest_node)

routes = {}
routes["Shortest"] = compute_route(orig_node, dest_node, "length")
routes["Fastest"]  = compute_route(orig_node, dest_node, "travel_time")
routes["Scenic"]   = compute_scenic_route(orig_node, dest_node)
routes["Safe"]     = compute_safe_route(orig_node, dest_node)

# Print summary
for k, rn in routes.items():
    if rn and len(rn) > 1:
        L, T = route_metrics(rn)
        print(f"{k}: nodes={len(rn)} length_m={L:.0f} time_min={T/60:.1f}")
    else:
        print(f"{k}: route not found")

# Choose route for animation
anim_choice = ANIMATION_ROUTE_CHOICE if ANIMATION_ROUTE_CHOICE in routes and routes[ANIMATION_ROUTE_CHOICE] else None
if not anim_choice:
    # fallback priority
    for p in ("Fastest","Shortest","Scenic","Safe"):
        if routes.get(p):
            anim_choice = p
            break
print("Animation route selected:", anim_choice)

# Prepare coordinates for drawing/animation
route_coords_by_name = {k: coords_from_nodes(rn) if rn else [] for k, rn in routes.items()}
anim_route_coords = route_coords_by_name.get(anim_choice, [])

# ---------------------------
# 5) Build folium map
# ---------------------------
m = folium.Map(location=ORIGIN, zoom_start=13, control_scale=True)
# Base layers
folium.TileLayer("OpenStreetMap", name="OSM Streets", attr="OpenStreetMap").add_to(m)
folium.TileLayer("Stamen Terrain", name="Stamen Terrain", attr="Stamen Terrain").add_to(m)
folium.TileLayer("Stamen Toner", name="Stamen Toner (B&W)", attr="Stamen Toner").add_to(m)
folium.TileLayer("Stamen Watercolor", name="Watercolor", attr="Stamen Watercolor").add_to(m)
folium.TileLayer("CartoDB positron", name="Light", attr="CartoDB Positron").add_to(m)
folium.TileLayer("CartoDB dark_matter", name="Dark Mode", attr="CartoDB Dark Matter").add_to(m)

# OpenTopoMap
folium.TileLayer(
    tiles="https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png",
    attr="OpenTopoMap",
    name="Topographic"
).add_to(m)

# ESRI Satellite & Terrain
folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
    name="ESRI Satellite",
    attr="Esri"
).add_to(m)

folium.TileLayer(
    tiles="https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}",
    name="ESRI Terrain",
    attr="Esri Topo"
).add_to(m)

# Google Hybrid
folium.TileLayer(
    tiles="http://{s}.google.com/vt/lyrs=y&hl=en&x={x}&y={y}&z={z}",
    name="Google Hybrid",
    attr="Google",
    subdomains=['mt0','mt1','mt2','mt3'],
    max_zoom=20
).add_to(m)

# Plugins
MiniMap(toggle_display=True).add_to(m)
MeasureControl().add_to(m)
Fullscreen().add_to(m)
MousePosition(position='bottomright', separator=' | ', prefix='Lat/Lon:').add_to(m)

# Layer control (after adding tile layers)
folium.LayerControl(collapsed=False).add_to(m)

# ---------------------------
# 6) Draw all computed routes (glow + main)
# ---------------------------
color_map = {"Shortest":"blue", "Fastest":"green", "Scenic":"purple", "Safe":"orange"}
for name, coords in route_coords_by_name.items():
    if not coords:
        continue
    # Glow (thick, translucent)
    folium.PolyLine(coords, color="#00FFFF", weight=12, opacity=0.25).add_to(m)
    # Main line
    folium.PolyLine(coords, color=color_map.get(name,"black"), weight=5, opacity=0.95, tooltip=name).add_to(m)
    # midpoint marker with stats
    mid = coords[len(coords)//2]
    L, T = route_metrics(routes[name]) if routes.get(name) else (0,0)
    folium.Marker(mid, popup=f"{name}<br>Length: {L:.0f} m<br>Time: {T/60:.1f} min",
                  icon=folium.DivIcon(html=f"""<div style="font-size:11px;color:{color_map.get(name)};">●</div>""")
                 ).add_to(m)

# Start / End markers
folium.Marker(ORIGIN, tooltip="Origin", icon=folium.Icon(color="darkgreen", icon="play")).add_to(m)
folium.Marker(DESTINATION, tooltip="Destination", icon=folium.Icon(color="red", icon="flag")).add_to(m)

# ---------------------------
# 7) Animation: moving vehicle along chosen route
# ---------------------------
if anim_route_coords and len(anim_route_coords) > 1:
    # Build timestamped features (one second step per point)
    features_anim = []
    for i, (lat, lon) in enumerate(anim_route_coords):
        # create ISO timestamp increasing by seconds (arbitrary date)
        features_anim.append({
            "type": "Feature",
            "geometry": {"type": "Point", "coordinates": [lon, lat]},
            "properties": {
                "time": f"2025-01-01T00:{(i%60):02d}:00",
                "icon": "marker",
                "iconstyle": {
                    # a small car icon; you may change to any accessible URL
                    "iconUrl": "https://cdn-icons-png.flaticon.com/512/743/743922.png",
                    "iconSize": [32,32],
                    "iconAnchor": [16,16]
                }
            }
        })
    TimestampedGeoJson(
        {"type":"FeatureCollection", "features": features_anim},
        period="PT1S",
        add_last_point=True,
        auto_play=True,
        loop=True,
        max_speed=1,
        loop_button=True,
        date_options="HH:mm:ss",
        duration="PT1M"
    ).add_to(m)
else:
    print("Animation route empty or not found; skipping animation.")

# ---------------------------
# 8) Small legend
# ---------------------------
legend_html = """
<div style="position: fixed;
     bottom: 60px; left: 10px; width: 200px; height: 140px;
     border:2px solid grey; z-index:9999; font-size:13px;
     background:white; padding:8px;">
<b>AI Routes</b><br>
&nbsp;<i style="color:blue;">●</i>&nbsp;Shortest<br>
&nbsp;<i style="color:green;">●</i>&nbsp;Fastest<br>
&nbsp;<i style="color:purple;">●</i>&nbsp;Scenic<br>
&nbsp;<i style="color:orange;">●</i>&nbsp;Safe<br>
<br><b>Animation:</b> {anim}
</div>
""".format(anim=anim_choice or "None")
m.get_root().html.add_child(folium.Element(legend_html))

# ---------------------------
# 9) Save & open
# ---------------------------
m.save(OUTFILE)
print("Saved:", OUTFILE)
try:
    webbrowser.open('file://' + os.path.realpath(OUTFILE))
except Exception:
    print("Could not auto-open — open the file manually in a browser.")

# Return map object (for Jupyter display)
m

Loading graph for: Srinagar, Jammu and Kashmir, India
Nodes: 21806 Edges: 50106
Edge speeds & travel times added.
Origin node: 6316628170 Destination node: 700171900
Shortest: nodes=139 length_m=8139 time_min=14.3
Fastest: nodes=144 length_m=9365 time_min=10.7
Scenic: nodes=77 length_m=19700 time_min=22.5
Safe: nodes=77 length_m=19700 time_min=22.5
Animation route selected: Fastest
Saved: ai_routes_final.html
